For all workers that work out of the home, the telecommute models predicts the level of telecommuting. The model alternatives are the frequency of telecommuting in days per week (0 days, 1 day, 2 to 3 days, 4+ days).

Telecommuting is defined as workers who work from home instead of going to work. It only applies to workers with a regular workplace outside of home.

In [1]:
import os
import toml
import pandas as pd
from validation_scripts import util
import validation_data_input
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"



In [2]:
# get shared data
%store -r validation_data

#config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

per_data = validation_data.persons_data_uncloned.copy()
land_use = validation_data.land_use[['zone_id', 'TAZ', 'county_id']].copy()

# delete shared data
del validation_data

per_data = per_data.merge(land_use,how="left",left_on='home_zone_id',right_on='zone_id')
per_data = per_data.loc[(per_data['pemploy']<3) & (~per_data['work_from_home'])]

survey data and unweighted survey inputs are not ready at the moment

In [3]:
df_plot = per_data.groupby(['source','telecommute_frequency'])[['person_weight']].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['person_weight']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="telecommute_frequency", y="percentage", color="source",barmode="group", template="simple_white",
             # color_discrete_sequence=config['psrc_color'],
             title="telecommute frequency<br>for workers with usual work location outside of home",
             category_orders={"telecommute_frequency": ["No_Telecommute", "1_day_week", "2_3_days_week", "4_days_week"]})
fig.update_layout(height=400, width=700, font=dict(size=11), yaxis=dict(tickformat=".2%"))
fig.show()

- worker count by telecommute frequency

In [4]:
per_data[['telecommute_frequency','source']].value_counts()

telecommute_frequency  source           
No_Telecommute         model results        989270
2_3_days_week          model results        312769
4_days_week            model results        199718
1_day_week             model results         89728
No_Telecommute         survey data            1678
                       unweighted survey      1678
2_3_days_week          survey data             699
                       unweighted survey       699
4_days_week            survey data             499
                       unweighted survey       499
1_day_week             survey data             208
                       unweighted survey       208
dtype: int64

In [5]:
# auto ownership in Income groups
def plot_segments(df:pd.DataFrame, var:str, title_cat:str,sub_name:str):
    # print(f"n=\n"
    #       f"{df.loc[df['source']=='model results',var].value_counts()[df[var].sort_values().unique()]}")
    df_plot = df.groupby(['source',var,'telecommute_frequency'])['person_weight'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source',var], group_keys=False)['person_weight'].\
        apply(lambda x: 100 * x / float(x.sum()))

    fig = px.bar(df_plot, x="telecommute_frequency", y="percentage", color="source",
                 facet_col=var, barmode="group",template="simple_white",
                 title="Work From Home by "+ title_cat)
    fig.for_each_annotation(lambda a: a.update(text = sub_name + "=<br>" + a.text.split("=")[-1]))
    fig.update_xaxes(title_text="n of work from home")
    fig.update_layout(height=400, width=800, font=dict(size=11))
    fig.show()

In [6]:
plot_segments(per_data,'county_id','County', 'Income')

In [7]:
fig = util.plot_segments(per_data, 'telecommute_frequency', 'county_id', 'Telecommute frequency', 'County', 'County')
fig.show()